In [ ]:
# The Ising model is a simple model for ferromagnetism. Atoms with spin
# S ∈ {−1, 1} are given fixed locations and their spin interactions are limited
# to nearest neighbours. The total energy (Hamiltonian) of the system is therefore

# H = -sum_<i,j> (S_iS_j)

# where <i,j> denotes neighbouring sites such that each interaction is only
# counted once.
# In two dimensions, the simplest Ising model looks like

# s11 s21 s31 s41
# s12 s22 s32 s42
# s13 s23 s33 s43
# s14 s24 s34 s44

# where, for instance, the neighbours of s22 are s21, s12, s32, s23.

# Make a function that takes an N × N matrix S and calculates H
# assuming a periodic system.

# a
def calculate_H(S):
    N = len(S)
    H = 0
    
    for i in range(N):
        for j in range(N):
            neighbor_sum = (
                S[i][(j + 1 ) % N] +  # Right neighbor
                S[i][(j - 1 ) % N] +  # Left neighbor
                S[(i + 1 ) % N][j] +  # Bottom neighbor
                S[(i - 1 ) % N][j]  # Top neighbor
            )
            H += -S[i][j] * neighbor_sum
    
    return H

# example
S = [
    [1, -1, 1, -1],
    [-1, 1, -1, 1],
    [1, -1, 1, -1],
    [-1, 1, -1, 1]
]

H = calculate_H(S)
print("H:", H)





In [ ]:
# In the lowest possible energy state all spins are aligned, i.e. they all equal
# 1 or −1, which corresponds to full magnetisation. However, fluctuations
# at finite temperatures will drive the system away from this minimum.
# The likelihood of a given spin configuration is given by the Boltzmann
# distribution

# p(S) = (e^(-H/T))/Z

# where T is the temperature and Z is the partition function. The magneti-
# sation of a specific spin configuration is simply

# m(S) = <S_i>

# Make a function that calculates the magnetisation.

# b
import numpy as np

def calculate_magnetization(S, T):
    N = len(S)
    partition_function = 0
    magnetization = 0
    H = 0
    for i in range(N):
        for j in range(N):
            neighbor_sum = (
                S[i, (j + 1) % N]
                + S[(i + 1) % N, j]
                + S[i, (j - 1) % N]
                + S[(i - 1) % N, j]
            )
            H += -S[i, j] * neighbor_sum
            partition_function += np.exp(-H / T)
            magnetization += S[i, j]

    magnetization /= (N * N)  # Normalize by the number of spins

    return magnetization

# Example spin configuration for a 4x4 lattice
spin_configuration = np.array([[1, -1, 1, -1],
                               [-1, 1, -1, 1],
                               [1, -1, 1, -1],
                               [-1, 1, -1, 1]])

temperature = 2.0  # Example temperature

magnetization = calculate_magnetization(spin_configuration, temperature)
print("Magnetization:", magnetization)



In [ ]:
# We shall be interested in the time-averaged magnetisation as a function
# temperature T. We will use the Metropolis–Hastings algorithm to simulate
# the time evolution of the system.

# Make a function that calculates the change in energy ΔE from a
# spin flip at a given location i.

# c
def calculate_delta_energy(S, i, j):
    N = len(S)
    neighbor_sum = (
        S[i, (j + 1) % N]
        + S[(i + 1) % N, j]
        + S[i, (j - 1) % N]
        + S[(i - 1) % N, j]
    )
    delta_energy = 2 * S[i, j] * neighbor_sum
    return delta_energy

import numpy as np

# Example spin configuration for a 4x4 lattice
spin_configuration = np.array([[1, -1, 1, -1],
                               [-1, 1, -1, 1],
                               [1, -1, 1, -1],
                               [-1, 1, -1, 1]])

i, j = 2, 2  # Example coordinates for spin flip

delta_energy = calculate_delta_energy(spin_configuration, i, j)
print("Change in energy (ΔE) from spin flip:", delta_energy)


In [ ]:
# Make a function that randomly accepts or rejects a spin flip based
# on ΔE with probability a = min(1, e^(−ΔE/T)).

# d
import numpy as np

def metropolis_hastings_acceptance(delta_energy, temperature):
    probability = min(1, np.exp(-delta_energy / temperature))
    return np.random.rand() < probability

# Example change in energy and temperature
delta_energy = 2
temperature = 1.5

accept = metropolis_hastings_acceptance(delta_energy, temperature)
if accept:
    print("Spin flip accepted!")
else:
    print("Spin flip rejected.")


In [ ]:
# To take a Metropolis–Hastings step, a random random spin is chosen
# and flipped according to the above rule.

# Start with a random initialisation of S with N ≥ 5 depending on
# the speed of your code (N = 1000 should be possible with numba). Let
# the system run for 1000 N^2 time steps to equilibrium the system. Plot the
# spin configuration (plt.imshow with interpolation=’nearest’) at
# different times during the simulation. Comment on the result.

# e
import numpy as np
import matplotlib.pyplot as plt
import numba

# Function to calculate the change in energy
@numba.jit
def calculate_delta_energy(S, i, j):
    N = len(S)
    neighbor_sum = (
        S[i, (j + 1) % N]
        + S[(i + 1) % N, j]
        + S[i, (j - 1) % N]
        + S[(i - 1) % N, j]
    )
    delta_energy = 2 * S[i, j] * neighbor_sum
    return delta_energy

# Function for Metropolis-Hastings acceptance
@numba.jit
def metropolis_hastings_acceptance(delta_energy, temperature):
    probability = min(1, np.exp(-delta_energy / temperature))
    return np.random.rand() < probability

# Initialize the lattice with random spins
def initialize_lattice(N):
    return np.random.choice([-1, 1], size=(N, N))

# Simulate the evolution of the system
def simulate_ising_model(N, temperature, steps):
    S = initialize_lattice(N)
    equilibration_steps = 1000 * N * N

    for step in range(steps + equilibration_steps):
        i, j = np.random.randint(N), np.random.randint(N)
        delta_energy = calculate_delta_energy(S, i, j)
        if metropolis_hastings_acceptance(delta_energy, temperature):
            S[i, j] *= -1

        if step % (steps // 10) == 0:
            plt.imshow(S, cmap='binary', interpolation='nearest')
            plt.title(f'Step {step}')
            plt.show()

# Parameters
N = 50  # Size of the lattice
temperature = 2.0
steps = 50000  # Total steps

# Run the simulation
simulate_ising_model(N, temperature, steps)




In [ ]:
# After initialisation let the system evolve another 1000 N^2 time
# steps, and store at each 100th time step the energy and the mag-
# netisation. Do this for temperatures between 0.1 and 5.0 (e.g.
# np.linspace(0.1, 5.0, 100)) and plot the average absolute mag-
# netisation as a function of temperature.

# f
import numpy as np
import matplotlib.pyplot as plt
import numba

# Function to calculate the change in energy
@numba.jit
def calculate_delta_energy(S, i, j):
    N = len(S)
    neighbor_sum = (
        S[i, (j + 1) % N]
        + S[(i + 1) % N, j]
        + S[i, (j - 1) % N]
        + S[(i - 1) % N, j]
    )
    delta_energy = 2 * S[i, j] * neighbor_sum
    return delta_energy

# Function for Metropolis-Hastings acceptance
@numba.jit
def metropolis_hastings_acceptance(delta_energy, temperature):
    probability = min(1, np.exp(-delta_energy / temperature))
    return np.random.rand() < probability

# Initialize the lattice with random spins
def initialize_lattice(N):
    return np.random.choice([-1, 1], size=(N, N))

# Simulate the evolution of the system and collect data
def simulate_ising_model(N, temperature, steps, collect_steps):
    S = initialize_lattice(N)
    equilibration_steps = 1000 * N * N
    energy_data = []
    magnetization_data = []

    for step in range(steps + equilibration_steps):
        i, j = np.random.randint(N), np.random.randint(N)
        delta_energy = calculate_delta_energy(S, i, j)
        if metropolis_hastings_acceptance(delta_energy, temperature):
            S[i, j] *= -1

        if step >= equilibration_steps and step % collect_steps == 0:
            energy = -np.sum(
                S * (
                    np.roll(S, 1, axis=0)
                    + np.roll(S, -1, axis=0)
                    + np.roll(S, 1, axis=1)
                    + np.roll(S, -1, axis=1)
                )
            )
            magnetization = np.abs(np.sum(S))
            energy_data.append(energy)
            magnetization_data.append(magnetization)

    return energy_data, magnetization_data

# Parameters
N = 50  # Size of the lattice
steps = 50000  # Total steps
collect_steps = 100  # Collect data every 100th step

# Temperatures to simulate
temperatures = np.linspace(0.1, 5.0, 100)

# Collect data for each temperature
average_magnetizations = []

for temperature in temperatures:
    energy_data, magnetization_data = simulate_ising_model(N, temperature, steps, collect_steps)
    average_magnetizations.append(np.mean(magnetization_data))

# Plotting
plt.plot(temperatures, average_magnetizations, marker='o', linestyle='-', color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Absolute Magnetization')
plt.title('Ising Model: Average Absolute Magnetization vs Temperature')
plt.grid(True)
plt.show()



In [ ]:
# You should find that the system spontaneously magnetises at low temper-
# ature. In particular, there is a phase transition at a critical temperature T_c
# below which the material is ferromagnetic.

# Determine T_c as best you can. What ways could you improve your
# estimate?

# g